# Import Required Libraries
Import necessary libraries such as random, matplotlib, and numpy.

In [29]:
# Import Required Libraries
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Define Helper Functions
Define helper functions including show(), line_intersection(), calculate_z_coordinate(), and calculate_inner_point().

In [30]:
def connect_points_in_order(n):
    """
    Connect points in the given order and return the lines connecting them.
    """
    return [(i, i + 1) for i in range(n - 1)] + [(n - 1, 0)]
def connect_all_points(n):
    """
    Connect every point to every other point and return the lines connecting them.
    """
    return [(i, j) for i in range(n) for j in range(i + 1, n)]

In [122]:
import math
# Define Helper Functions

def line_intersection(A, B, C, D):
    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    xdiff = (A[0] - B[0], C[0] - D[0])
    ydiff = (A[1] - B[1], C[1] - D[1])

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('Lines do not intersect')

    d = (det(A, B), det(C, D))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y
def first(x,y,z, line):
    return (x[line[0]], y[line[0]], z[line[0]])
def second(x,y,z, line):
    return (x[line[1]], y[line[1]], z[line[1]])
def first_2d(x,y, line):
    return (x[line[0]], y[line[0]])
def second_2d(x,y, line):
    return (x[line[1]], y[line[1]])

def show_2d(x, y, z, lines):
    # ax_2d.cla()
    # ax_2d.scatter(x, y, c='r', marker='o')

    # # Annotate each point with its index
    # for idx, (x_coord, y_coord) in enumerate(zip(x, y)):
    #     ax_2d.annotate(idx, (x_coord, y_coord), textcoords="offset points", xytext=(0, 10), ha='center')

    vertices = [*zip(x,y)]
    connectivities = []
    def calculate_z_coordinate(intersection_point,A,B):
        z_A = A[2]
        z_B = B[2]
        if A[0] != B[0]:
            return z_A + (z_B - z_A) * ((intersection_point[0] - A[0]) / (B[0] - A[0]))
        else:
            return z_A + (z_B - z_A) * ((intersection_point[1] - A[1]) / (B[1] - A[1]))

    overlapping_lines = [[] for _ in range(len(lines))]
    for i in range(len(lines)):
        for j in range(len(lines)):
            if i == j:
                continue
            if(lines[i][0] in lines[j] or lines[i][1] in lines[j]):
                continue
            def ccw(A, B, C):
                return (C[1] - A[1]) * (B[0] - A[0]) >= (B[1] - A[1]) * (C[0] - A[0])
            A = first(x,y,z, lines[i])
            B = second(x,y,z, lines[i])
            C = first(x,y,z, lines[j])
            D = second(x,y,z, lines[j])
            if ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D):
                inter = line_intersection(A, B, C, D)
                z1 = calculate_z_coordinate(inter, A,B)
                z2 = calculate_z_coordinate(inter, C,D)
                if z1 > z2:
                    overlapping_lines[j].append(inter)
                else:
                    overlapping_lines[i].append(inter)

    for i, line in enumerate(overlapping_lines):
        if first_2d(x,y,lines[i])[0] > first_2d(x,y,lines[i])[1]:
            overlapping_lines[i] = sorted(line, key=lambda point: point[0], reverse=True)
        else:
            overlapping_lines[i] = sorted(line, key=lambda point: point[0])

    for i, l in enumerate(overlapping_lines):
        start_point = first_2d(x,y, lines[i])
        end_point = second_2d(x,y, lines[i])
        #connectivities.append(lines[i])
        
        if len(l) == 0:
            connectivities.append(lines[i])
            #ax_2d.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='b', linewidth=1)
        else:
            def calculate_inner_point(A, B, l):
                size = math.sqrt((A[0]-B[0]) ** 2 + (A[1]-B[1]) ** 2)
                if(size == 0):
                    return A
                l = 0.2
                #if(l < size):
                #    return (A[0] + l * (B[0] - A[0]) / size, A[1] + l * (B[1] - A[1]) / size)
                #else:
                return (A[0] + l * (B[0] - A[0])/size, A[1] + l * (B[1] - A[1])/size)
            t01 = calculate_inner_point(l[0],start_point, 0.2)
            t02 = calculate_inner_point(l[len(l) - 1], end_point, 0.2)
            if(start_point[0] < t01[0]<l[0][0] or start_point[0] > t01[0]>l[0][0]):
                idx = len(vertices)
                vertices.append(t01)
                connectivities.append((idx,lines[i][0]))
            #ax_2d.plot((start_point[0], t01[0]), (start_point[1], t01[1]), color='b', linewidth=1)
            for index in range(len(l) - 1):
                t1 = calculate_inner_point(l[index], l[index + 1], 0.2)
                t2 = calculate_inner_point(l[index + 1],l[index], 0.2)
                #check validity
                if((l[index][0] < l[index + 1][0] and t1[0] < t2[0]) or (l[index][0] > l[index + 1][0] and t1[0] > t2[0])):
                    idx = len(vertices)
                    vertices.append(t1)
                    vertices.append(t2)
                    connectivities.append((idx, idx+1))
                    #ax_2d.plot((t1[0], t2[0]), (t1[1], t2[1]), color='b', linewidth=1)
            if(end_point[0] < t02[0]<l[len(l) - 1][0] or end_point[0] > t02[0]>l[len(l) - 1][0]):
                idx = len(vertices)
                vertices.append(t02)
                connectivities.append((idx, lines[i][1]))
            #ax_2d.plot((t02[0], end_point[0]), (t02[1], end_point[1]), color='b', linewidth=1)
    #ax_2d.set_title('2D Projection of 3D Points')
    #ax_2d.set_axis_off()
    #ax_2d.set_xlim(-3, 3)
    #ax_2d.set_ylim(-3, 3)
    return vertices, connectivities
def show_3d(x,y,z,lines,ax1):
    ax1.cla()
    ax1.scatter(x, y, z, c='r', marker='o')
    ax1.set_title('3D Projection of Points')

    # Annotate each point with its index
    for idx, (x_coord, y_coord, z_coord) in enumerate(zip(x, y, z)):
        ax1.text(x_coord, y_coord, z_coord, '%d' % idx, size=10, zorder=1, color='k')

    ax1.set_axis_off()
    # Connect the points with lines
    ax1.plot(x, y, z, color='b', linewidth=1)
    for (i,j) in lines:
        ax1.plot([x[i], x[j]], [y[i], y[j]], [z[i], z[j]], color='b', linewidth=1)

# Generate Random Points in 3D
Generate random points in 3D space and sort them to follow the surfaces.

In [32]:
def rotation_matrix_x(theta):
    """
    Create a rotation matrix for rotating around the x-axis by theta radians.
    """
    return np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])

def rotation_matrix_y(theta):
    """
    Create a rotation matrix for rotating around the y-axis by theta radians.
    """
    return np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])

def rotation_matrix_z(theta):
    """
    Create a rotation matrix for rotating around the z-axis by theta radians.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

In [119]:
import moderngl
import numpy as np
from matplotlib.animation import FuncAnimation

# Initialize ModernGL context
ctx = moderngl.create_standalone_context()

# Create framebuffer
fbo = ctx.framebuffer(color_attachments=[ctx.texture((800, 600), 4)])

# Create vertex buffer and program
prog = ctx.program(
    vertex_shader="""
    #version 330
    in vec3 in_vert;
    void main() {
        gl_Position = vec4(in_vert, 1.0);
    }
    """,
    fragment_shader="""
    #version 330
    out vec4 fragColor;
    void main() {
        fragColor = vec4(0.0, 0.0, 1.0, 1.0);
    }
    """
)

# Create animation frame generator
def create_animation_frame(points,conn, num_frames=100, show_point = True):
    theta = 0
    
    while theta < 2 * np.pi:
        # Create rotation matrix by combining rotations around all axes
        # Define 2D vertices and connections
        x, y, z = zip(*[np.dot(rotation_matrix_x(1.1*theta), np.dot(rotation_matrix_y(theta),np.dot(rotation_matrix_z(theta),point))) for point in points])
        vertices, connections = show_2d(x, y, z, conn)
        projected_vertices = [*zip(x,y)]
        #vertices = [(0,0),(1,1),(-1,1), (-1,-1)]
        vertices = np.array(vertices, dtype='f4').flatten() / 3
        projected_vertices = np.array(projected_vertices, dtype='f4').flatten() / 3
        connections = np.array(connections, dtype='i4').flatten()
        #print(vertices, connections)
        
        # Create vertex buffer for connections
        vbo_vertices = ctx.buffer(vertices.tobytes())
        
        # Create vertex buffer for projected vertices
        vbo_projected = ctx.buffer(projected_vertices.tobytes())
        
        # Create index buffer
        ibo = ctx.buffer(connections.tobytes())

        # Create vertex array object for connections
        vao_connections = ctx.vertex_array(prog, [(vbo_vertices, '2f', 'in_vert')], ibo)
        
        # Create vertex array object for projected vertices
        vao_projected = ctx.vertex_array(prog, [(vbo_projected, '2f', 'in_vert')])

        vao_vertices = ctx.vertex_array(prog, [(vbo_vertices, '2f', 'in_vert')])
        # Clear framebuffer
        fbo.use()
        ctx.clear(1.0, 1.0, 1.0, 1.0)

        # Render lines for connections
        vao_connections.render(moderngl.LINES)
        
        # Render points for projected vertices with larger size
        if(show_point):
            ctx.point_size = 3.0
            vao_projected.render(moderngl.POINTS)
            ctx.point_size = 1.0
            vao_vertices.render(moderngl.POINTS)
        # Read pixels
        data = fbo.read(components=3)
        frame = np.frombuffer(data, dtype=np.uint8).reshape((600, 800, 3))
        theta += 2 * np.pi / num_frames
        yield frame
three_points = [
    (1, 1, 1), (-1, -1, 1), (-1, 1, -1), (1, -1, -1)
]
# Define the vertices of a cube
four_points = [
    (-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
    (1, 1, -1), (1, 1, 1), (1, -1, -1), (1, -1, 1), 
]


phi = (1 + np.sqrt(5)) / 2  # Golden ratio
five_points = [
    (-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
    (1, -1, -1), (1, -1, 1), (1, 1, -1), (1, 1, 1),
    (0, -1/phi, -phi), (0, -1/phi, phi), (0, 1/phi, -phi), (0, 1/phi, phi),
    (-1/phi, -phi, 0), (-1/phi, phi, 0), (1/phi, -phi, 0), (1/phi, phi, 0),
    (-phi, 0, -1/phi), (-phi, 0, 1/phi), (phi, 0, -1/phi), (phi, 0, 1/phi)
]
# Create animation frames
frames = list(create_animation_frame(four_points, connect_all_points(len(four_points))))

# Update the animation
def update(frame):
    im.set_array(frames[frame])
    return [im]

# Create animation
fig, ax = plt.subplots()
im = ax.imshow(frames[0])
ani = FuncAnimation(fig, update, frames=len(frames), interval=50)
ani.save('knot_theory_visualization.mp4')
plt.close()


In [ ]:
def polygonalize(f, delta, total_length):
    return [f(i * delta) for i in range(int(total_length / delta))]

def f(s):
    """
    Parametric equation for a trefoil knot.
    """
    x = np.sin(s) + 2 * np.sin(2 * s)
    y = np.cos(s) - 2 * np.cos(2 * s)
    z = -np.sin(3 * s)
    return (x, y, z)

data = polygonalize(f, 0.05, 2 * np.pi)
frames = list(create_animation_frame(data, connect_points_in_order(len(data)), show_point=False))

# Update the animation
def update(frame):
    im.set_array(frames[frame])
    return [im]

# Create animation
fig, ax = plt.subplots()
im = ax.imshow(frames[0])
ani = FuncAnimation(fig, update, frames=len(frames), interval=50)
ani.save('knot_theory_visualization.mp4')
plt.close()
